
Inputs: HOEP and Load Data

Decision variables: PCDDR PCEA (24 x 6)

Constraints:         
        
        PNT = PDLL + PDLS - PCEA           (7)
        
        NTB = 0 if MEC <= 0                (5)
        NTB = MEC if MEC > 0               (5)
        
        MEC = sum(30* (sum(PNT * HOEP)))   (6)
        REV = sum(PCDDR*KRDDR)
        
        Profits = REV - MEC
        
        PD = PNT + PSB                     (13.5)
        
        0 <= ESB <= 1000
        
        1000 - ESB >= 0 & ESB >=0 
        
        ESB(t) = PCEA + ESB(t-1) - PCDDR
        
        -5000 + PCDDR <= PSB <= 5000       (10)
        
        5000 - PSB >= 0 
        
        PSB - PCDDR + 5000 >= 0
        
        |PCDDR| + |PCEA| <= 5000           (11)
        
        5000 - (|PCDDR| + |PCEA|) >= 0

In [2]:
import numpy as np
import pandas as pd
import mosek
from scipy.optimize import minimize

In [2]:
## Read in csv file from IESO website, ***fill blank entries with 0*** (temp)
df = pd.read_csv("http://reports.ieso.ca/public/PriceHOEPPredispOR/PUB_PriceHOEPPredispOR_2019.csv").fillna(0)

df.rename(columns=df.iloc[2], inplace=True)         ## Set headers to the proper ones row 4
df = df[3:]
df.reset_index(inplace=True, drop=True)             ## Reset indices to proper values

#df.dropna(inplace=True)


## Convert columns to suitable data types
df['Date'] = pd.to_datetime(df['Date'])
df = df.astype({'Hour':int, 'HOEP':float, 'Hour 1 Predispatch': float, 'Hour 2 Predispatch': float, 'Hour 3 Predispatch':float, 'OR 10 Min Sync':float, 'OR 10 Min non-sync':float, 'OR 30 Min':float})

## Split the date into year, month, day
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df.drop(['Date'], axis=1, inplace=True)

## Rearrange them so they appear at the beginning (not necessary, only intermediate step for you to visualize)
date = ['Year', 'Month', 'Day']
df = df[date + [c for c in df if c not in date]]

## Create new dataframe for final data values
data = pd.DataFrame()

## Iterate through the months of the year specified from CSV file
## and iterate through the hours to get monthly average for that specific hour
for month in range(1,13):
    average = []
    df_month = df.loc[df['Month'] == month]

    for hour in range(1,25):
        h = df_month.loc[df['Hour'] == hour]
        average.append((h['HOEP'].sum()/h.shape[0])/100)  ## Cents to Dollars
    data['Month ' + str(month)] = average

## Set index to proper hours
data.index = range(1, len(data)+1)
data.index.name = 'Hour'


hoep_data_6months = pd.DataFrame()
for i in range(0, 12, 2):
    hoep_data_6months[data.columns[i] + ' & ' + data.columns[i+1]] = (data[data.columns[i]] + data[data.columns[i+1]])/2
hoep_data_6months

,Month 1 & Month 2,Month 3 & Month 4,Month 5 & Month 6,Month 7 & Month 8,Month 9 & Month 10,Month 11 & Month 12
Hour,,,,,,
1,0.203923,0.160390,0.017717,0.102176,0.022171,0.138881
2,0.189694,0.163950,-0.022535,0.075305,0.025111,0.138952
3,0.177902,0.157672,-0.024723,0.066639,0.018327,0.117141
4,0.191661,0.148313,-0.019338,0.062776,0.015657,0.119464
5,0.177496,0.140120,-0.012078,0.058413,0.021688,0.099382
6,0.174045,0.184558,0.016359,0.073076,0.072808,0.084569
7,0.217018,0.259519,0.042125,0.116798,0.111915,0.130491
8,0.368089,0.262885,0.100699,0.172197,0.095109,0.347484
9,0.303033,0.244769,0.068821,0.187535,0.136750,0.417087


In [3]:
load_data = pd.read_excel('load_data.xlsx', index_col=0)
#load_data

Inputs: HOEP and Load Data

Decision variables: PCEA (24 x 6)

Constraints:

    PNT = PDLL + PCEA                            (7)

    MEC = sum(30* (sum((PDLL + PCEA) * HOEP)))   (6)

    -5000 <= PCEA <= 5000                        (11)
    
    ESB_t = ESB_(t-1) + PCEA                     (16)
    

In [4]:
## x[0:24] = PCEA
## x[24:48] = ESB
## x[48:72] = PCDDR


def constraint1(x):
    for i in range(24):
        if (i == 0):
            x[24] = 0
        else:
            x[24+i] = x[24+i-1] + x[i]
    return x[0:24] + x[24:48] - x[48:72]
    
def constraint2(x):
    for i in range(24):
        if (i == 0):
            x[24] = 0
        else:
            x[24+i] = x[24+i-1] + x[i]
    return 10000 - (x[0:24]+ x[24:48]- x[48:72])
   
    

power = ((-5000, 5000),) * 24
storage = ((0, 10000),) * 24
DDR = ((0,5000),) * 24

KRDDR = 856.436/8760

##Objective
def MEC(x):           #      (    PDLL             +             PCEA     )          x        HOEP
    return -(KRDDR*sum(30*sum(np.array([x[48:72]]))) - \
            sum(30*sum((load_data.iloc[:,month].to_numpy() + np.array([x[0:24]])) * hoep_data_6months.iloc[:,month].to_numpy())))


x0 = np.array([np.ones(24), np.ones(24), np.ones(24)])
bounds = (power + storage + DDR)
cons1 = {'type': 'ineq', 'fun': constraint1}
cons2 = {'type': 'ineq', 'fun': constraint2}

cons = ([cons1, cons2])

pcea_solutions = []
esb_solutions = []
pcddr_solutions = []

mec = 0
for month in range(6):
    sol = minimize(MEC, x0, method='SLSQP',
                   bounds=bounds,
                   constraints=cons,
                   options= {'maxiter':300,'disp':True})
    pcea_solutions.append(sol.x[0:24])
    esb_solutions.append(sol.x[24:48])
    pcddr_solutions.append(sol.x[48:72])
    mec = mec + sol.fun
    
print(mec)



Optimization terminated successfully.    (Exit mode 0)
            Current function value: -100663.00128069334
            Iterations: 121
            Function evaluations: 8658
            Gradient evaluations: 117
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -280255.10928165657
            Iterations: 168
            Function evaluations: 12136
            Gradient evaluations: 164
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -361608.37616910617
            Iterations: 187
            Function evaluations: 13621
            Gradient evaluations: 184
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -217327.7507344823
            Iterations: 133
            Function evaluations: 9546
            Gradient evaluations: 129
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -304758.64679999027
            Iterations: 

In [5]:
df_sol = pd.concat([pd.DataFrame(data=pcea_solutions).transpose(), pd.DataFrame(data=esb_solutions).transpose(),\
                    pd.DataFrame(data=pcddr_solutions).transpose()], axis=1)

In [6]:
df_sol.columns = load_data.columns.values.tolist() + load_data.columns.values.tolist() + load_data.columns.values.tolist()
df_sol

,Month 1 & Month 2,Month 3 & Month 4,Month 5 & Month 6,Month 7 & Month 8,Month 9 & Month 10,Month 11 & Month 12,Month 1 & Month 2,Month 3 & Month 4,Month 5 & Month 6,Month 7 & Month 8,Month 9 & Month 10,Month 11 & Month 12,Month 1 & Month 2,Month 3 & Month 4,Month 5 & Month 6,Month 7 & Month 8,Month 9 & Month 10,Month 11 & Month 12
0,-1.908020e-11,-4.111767e-12,5000.000000,2.835781e-11,5000.000000,-8.858020e-11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.881448e-12,2.428280e-12,5000.0,0.000000,5000.0,6.341639e-11
1,3.116644e+03,2.500031e+03,5000.000000,2.867065e+03,5000.000000,2.500000e+03,3116.643897,2500.031023,5000.000000,2867.064558,5000.000000,2500.000000,5.000000e+03,5.000000e+03,5000.0,5000.000000,5000.0,5.000000e+03
2,4.506726e+03,5.000000e+03,5000.000000,3.664391e+03,5000.000000,3.037923e+03,7623.369618,7500.031023,10000.000000,6531.455950,10000.000000,5537.923493,5.000000e+03,5.000000e+03,5000.0,5000.000000,5000.0,5.000000e+03
3,2.636635e+03,3.749984e+03,2500.000000,4.127202e+03,2500.000000,2.833274e+03,10260.004206,11250.015511,12500.000000,10658.657628,12500.000000,8371.197642,5.000000e+03,5.000000e+03,5000.0,5000.000000,5000.0,5.000000e+03
4,2.369998e+03,1.874992e+03,1250.000000,2.170671e+03,1250.000000,3.314401e+03,12630.002103,13125.007756,13750.000000,12829.328814,13750.000000,11685.598821,5.000000e+03,5.000000e+03,5000.0,5000.000000,5000.0,5.000000e+03
5,1.184999e+03,9.374961e+02,625.000000,1.085336e+03,625.000000,1.657201e+03,13815.001051,14062.503878,14375.000000,13914.664407,14375.000000,13342.799410,5.000000e+03,5.000000e+03,5000.0,5000.000000,5000.0,5.000000e+03
6,5.924995e+02,-4.531252e+03,312.500000,5.426678e+02,312.500000,8.286003e+02,14407.500526,9531.251939,14687.500000,14457.332204,14687.500000,14171.399705,5.000000e+03,5.000000e+03,5000.0,5000.000000,5000.0,5.000000e+03
7,-4.703750e+03,-2.265626e+03,-4843.750000,2.713339e+02,156.250000,-2.660845e+03,9703.750263,7265.625969,9843.750000,14728.666102,14843.750000,11510.554481,5.000000e+03,5.000000e+03,5000.0,5000.000000,5000.0,5.000000e+03
8,9.784426e+02,-1.132813e+03,-2421.875000,1.223456e+02,-4921.875000,-3.886718e+03,10682.192859,6132.812985,7421.875000,14851.011725,9921.875000,7623.836528,5.000000e+03,5.000000e+03,5000.0,5000.000000,5000.0,3.737119e+03
9,-2.822441e+03,-5.664065e+02,3789.062500,3.041415e+01,-2460.937500,-1.311918e+03,7859.751826,5566.406492,11210.937500,14881.425878,7460.937500,6311.918264,5.000000e+03,5.000000e+03,5000.0,5000.000000,5000.0,5.000000e+03


In [7]:
df_sol.to_excel('Opt_Module_v1.0.xlsx')

FileCreateError: [Errno 13] Permission denied: 'Opt_Module_v1.0.xlsx'

In [ ]:
p = sol.x[0:24]

In [ ]:
e = sol.x[24:48]

In [ ]:
x_ax = np.arange(1,25)
x_ax

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, axs = plt.subplots(1,3)
fig.set_size_inches(20, 5)
axs[0].plot(x_ax, p)
axs[0].plot(x_ax, e)
axs[1].plot(x_ax, data_6months.iloc[:,0])
axs[0].legend(['PCEA', 'ESB'])
axs[2].plot(x_ax, load_data.iloc[:,0])

In [ ]:
import scipy
scipy.__version__